In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm

2025-07-28 15:55:21.126416: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-28 15:55:21.133319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753698321.141208   15755 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753698321.143596   15755 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753698321.149788   15755 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
DATA_PATH = '/home/smayan/Desktop/Cricket Pose Estimation /Data'
sequence_length = 30
min_sequences_per_class = 10

In [ ]:
actions = np.array(sorted([folder for folder in os.listdir(DATA_PATH) 
                          if os.path.isdir(os.path.join(DATA_PATH, folder))]))
print(f"Detected cricket shots: {actions}")
for action in actions:
    video_files = [f for f in os.listdir(os.path.join(DATA_PATH, action)) if f.endswith(('.mp4', '.avi', '.mov'))]
    video_files = video_files[:25]
    print(f"{action}: {len(video_files)} videos")

Detected cricket shots: ['Backfoot punch' 'Cover drive' 'Cut Shot' 'FBD' 'Flick'
 'Front Food defence' 'On Drive' 'Pull Shot' 'Reverse Sweep' 'Stance'
 'Straight Drive' 'Sweep' 'Uppercut' 'loft']
Backfoot punch: 19 videos
Cover drive: 25 videos
Cut Shot: 25 videos
FBD: 15 videos
Flick: 22 videos
Front Food defence: 25 videos
On Drive: 25 videos
Pull Shot: 25 videos
Reverse Sweep: 25 videos
Stance: 25 videos
Straight Drive: 25 videos
Sweep: 25 videos
Uppercut: 25 videos
loft: 25 videos


In [5]:
sequences = []
labels = []
label_map = {label: num for num, label in enumerate(actions)}

In [7]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        video_files = [f for f in os.listdir(action_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        video_files = video_files[:25]
        action_sequences = []

        for video_file in tqdm(video_files, desc=f"Processing {action}"):
            video_path = os.path.join(action_path, video_file)
            cap = cv.VideoCapture(video_path)
            total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
            stride = max(1, sequence_length // 4)

            for start_frame in tqdm(range(0, total_frames - sequence_length + 1, stride), 
                                    desc=f"Frames in {video_file}", leave=False):
                cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
                sequence = []

                for _ in range(sequence_length):
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frame = cv.resize(frame, (640, 480))
                    image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                    image.flags.writeable = False
                    results = holistic.process(image)
                    image.flags.writeable = True
                    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
                    if results.pose_landmarks:
                        pose = np.array([[res.x, res.y, res.z, res.visibility] 
                                         for res in results.pose_landmarks.landmark]).flatten()
                    else:
                        pose = np.zeros(33*4)
                    sequence.append(pose)

                if len(sequence) == sequence_length:
                    action_sequences.append(sequence)

            cap.release()

        while len(action_sequences) < min_sequences_per_class:
            if action_sequences:
                original_seq = np.array(action_sequences[len(action_sequences) % len(action_sequences)])
                noise = np.random.normal(0, 0.01, original_seq.shape)
                augmented_seq = original_seq + noise
                action_sequences.append(augmented_seq.tolist())

        for seq in action_sequences:
            sequences.append(seq)
            labels.append(label_map[action])
        print(f"Generated {len(action_sequences)} sequences for {action}")

I0000 00:00:1753698498.130242   15755 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1753698498.187942   17836 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 570.172.08), renderer: NVIDIA GeForce RTX 4070 SUPER/PCIe/SSE2
Processing Backfoot punch:   0%|          | 0/19 [00:00<?, ?it/s]W0000 00:00:1753698498.221536   17812 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753698498.242115   17823 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753698498.243460   17809 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753698498.243569   17813 inference_feedback_manager.cc:114] Feedback manager requires a model with a single s

Generated 112 sequences for Backfoot punch


Processing Cover drive: 100%|██████████| 25/25 [01:34<00:00,  3.76s/it]


Generated 132 sequences for Cover drive


Processing Cut Shot: 100%|██████████| 25/25 [01:20<00:00,  3.22s/it]


...

Generated 149 sequences for Sweep


Processing Uppercut: 100%|██████████| 25/25 [01:31<00:00,  3.66s/it]


Generated 126 sequences for Uppercut


Processing loft: 100%|██████████| 25/25 [01:46<00:00,  4.26s/it]

Generated 149 sequences for loft


In [26]:
X = np.array(sequences)
y = np.array(labels)
print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")


np.save('training_data.npy', X)
np.save('labels.npy', y)


Dataset shape: (1939, 30, 132)
Labels shape: (1939,)


In [27]:
X

array([[[ 5.64968348e-01,  2.49950171e-01,  3.38950045e-02, ...,
          7.92152047e-01,  1.12857200e-01,  1.97137430e-01],
        [ 5.64456105e-01,  2.49205589e-01,  1.35365976e-02, ...,
          7.91108251e-01,  7.04743341e-02,  2.10779011e-01],
        [ 5.63663423e-01,  2.49177113e-01,  1.81654505e-02, ...,
          7.89731920e-01,  7.93914273e-02,  2.19110340e-01],
        ...,
        [ 3.96243960e-01,  3.10435861e-01,  1.03527762e-01, ...,
          8.21304440e-01, -1.22316718e-01,  6.95255220e-01],
        [ 4.02454674e-01,  3.31085622e-01,  3.33393335e-01, ...,
          7.87335753e-01, -4.52745110e-01,  7.09456921e-01],
        [ 3.95582497e-01,  3.26448768e-01,  2.72976100e-01, ...,
          7.80987144e-01, -2.21305177e-01,  7.36743331e-01]],

       [[ 5.35325587e-01,  2.80128688e-01,  9.64741707e-02, ...,
          8.29995155e-01, -7.46523682e-03,  6.73663378e-01],
        [ 5.31645834e-01,  2.69057244e-01,  3.76071148e-02, ...,
          8.31327558e-01,  1.41000867e

In [9]:
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
y_categorical = to_categorical(y, num_classes=len(actions))

In [ ]:
x

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y
)


In [11]:
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = dict(enumerate(class_weights))

In [12]:
model = Sequential()

# TimeDistributed CNN - 1 block only
model.add(TimeDistributed(Conv1D(32, kernel_size=3, activation='relu'), input_shape=(sequence_length, X.shape[2], 1)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Dropout(0.2)))
model.add(TimeDistributed(Flatten()))
model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)))
model.add(LSTM(32, return_sequences=False, dropout=0.3, recurrent_dropout=0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(len(actions), activation='softmax'))

/home/smayan/Desktop/AI-ML-DS/AI-and-ML-Course/.conda/lib/python3.11/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1753699824.070778   15755 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9077 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


In [13]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = f'logs/cricket_model_{timestamp}'

In [14]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
callbacks = [
    TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq='epoch'),
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=10, min_lr=1e-7)
]

In [16]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/50


I0000 00:00:1753699830.828332  492448 cuda_dnn.cc:529] Loaded cuDNN version 91100


49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - accuracy: 0.0682 - loss: 2.6312 - val_accuracy: 0.1675 - val_loss: 2.4869 - learning_rate: 0.0010
Epoch 2/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.1439 - loss: 2.4772 - val_accuracy: 0.1856 - val_loss: 2.3079 - learning_rate: 0.0010
Epoch 3/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.1870 - loss: 2.2657 - val_accuracy: 0.3093 - val_loss: 1.9944 - learning_rate: 0.0010
Epoch 4/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.2627 - loss: 2.0395 - val_accuracy: 0.3608 - val_loss: 1.7872 - learning_rate: 0.0010
Epoch 5/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - accuracy: 0.3323 - loss: 1.8402 - val_accuracy: 0.4407 - val_loss: 1.6022 - learning_rate: 0.0010
Epoch 6/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.4058 - loss: 1.6375 - val_accuracy: 0.4613 - val_loss: 1.4491 - learning_rate: 0.0010
Epoch 7/50
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.4512 - loss: 1.5883 - val_accuracy: 0.5

In [17]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9433


In [18]:
model.save('cricket_pose_mode_simple_even.h5')
model.save('cricket_pose_model_simple_even.keras')
np.save('cricket_label_map.npy', label_map)

In [21]:
# Load trained model and label map
model = tf.keras.models.load_model('cricket_pose_mode_simple_even.h5')
label_map = np.load('cricket_label_map.npy', allow_pickle=True).item()
actions = list(label_map.keys())

# Variables for prediction
sequence = []
sequence_length = 30
threshold = 0.7

# Start webcam
cap = cv.VideoCapture('/home/smayan/Desktop/Cricket Pose Estimation /Model Training/WhatsApp Video 2025-07-27 at 16.12.07.mp4')

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize for consistent input
        frame = cv.resize(frame, (640, 480))

        # Detection
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
            )

        # Extract keypoints
        if results.pose_landmarks:
            keypoints = np.array([[res.x, res.y, res.z, res.visibility]
                                  for res in results.pose_landmarks.landmark]).flatten()
        else:
            keypoints = np.zeros(33*4)

        # Append to sequence
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]

        if len(sequence) == sequence_length:
            input_seq = np.expand_dims(np.array(sequence), axis=0)
            input_seq = input_seq.reshape(1, sequence_length, -1, 1)

            # Predict
            res = model.predict(input_seq, verbose=0)[0]
            predicted_action = actions[np.argmax(res)]
            confidence = np.max(res)

            # Show prediction
            if confidence > threshold:
                cv.putText(image, f'{predicted_action}: {confidence:.2f}',
                           (10, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show probabilities
            for i, (action, prob) in enumerate(zip(actions, res)):
                y_pos = 100 + i * 30
                cv.rectangle(image, (10, y_pos), (int(prob * 300) + 10, y_pos + 25), (0, 255, 0), -1)
                cv.putText(image, f'{action}: {prob:.2f}', (15, y_pos + 18),
                           cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        # Show output
        cv.imshow('Cricket Pose Estimation', image)

        # Quit
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()


I0000 00:00:1753700199.920077   15755 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1753700199.966096  531587 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 570.172.08), renderer: NVIDIA GeForce RTX 4070 SUPER/PCIe/SSE2
W0000 00:00:1753700200.001929  531564 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753700200.022952  531575 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753700200.023877  531564 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753700200.025036  531560 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000